In [1]:
import csv
import pandas as pd
from sklearn.linear_model import LogisticRegression 
from sklearn.preprocessing import LabelEncoder
LE=LabelEncoder()
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

In [2]:
segment1_workflow4=pd.read_csv('segment1_workflowS4.csv')
segment1_workflow4.CASE_ID.mode()

/Users/ab0707/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (31,37) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


0    1043915|389782|606944
dtype: object

In [3]:
segment1_workflow4.head()

,CASE_ID,WRKFLW_TSK_MAP_GROUPED,WRKFLW_TSK_MAP_DETAILED,PATH_CRDATETIME,ITEM_CRUSER,WRKFLW_ORG_TYPE_CRUSER,STATUS_ORDER,LATEST_STATUS,WRKFLW_PROCESS,QUERY_PART,...,ORIG_SPRD_CD,ORIG_SPRD_NM,SPK_CAT_TYPE_CD,BROAD_COURSE_LEVEL,DEGREETYPE,LIABILITY_CATEGORY_CD_RECODE,ORIG_LIABILITY_CATEGORY_CD_RECODE,OFFER_RELEASED_FG,OFFER_OUTCOME_CD,OFFER_CONDITIONS
0,804599|387970|604118,Release Offer,Release Offer,19/9/2018 10:26:34,KRIT,Faculty,9,N,3b. Offer stage,Part 2 - Application Line ID Workflow,...,MDYI,Mid Year Intake,MCW,HDCW,Full Degree,OF,OF,Y,$DEF,Y
1,804599|387970|604118,OC - Satisfied - Other,OC - Satisfied - Evidence of course completion,14/1/2019 11:09:39,KNAVIN,Academic Services,8,N,1. App to Assessment,Part 6 - Requirement Verify,...,MDYI,Mid Year Intake,MCW,HDCW,Full Degree,OF,OF,Y,$DEF,Y
2,804599|387970|604118,Defer,Defer,25/1/2019 1:28:07,KRIT,Faculty,7,N,4. Post offer,Part 2 - Application Line ID Workflow,...,MDYI,Mid Year Intake,MCW,HDCW,Full Degree,OF,OF,Y,$DEF,Y
3,844158|432073|669073,Release Offer,Release Offer,8/3/2019 11:31:03,KRIT,Faculty,7,N,3b. Offer stage,Part 2 - Application Line ID Workflow,...,MDYI,Mid Year Intake,MCW,HDCW,Full Degree,OF,OF,Y,$ACC,Y
4,844158|432073|669073,Manage Acceptance,Manage Acceptance,6/7/2019 1:21:43,844158,NaN,6,N,4. Post offer,Part 2 - Application Line ID Workflow,...,MDYI,Mid Year Intake,MCW,HDCW,Full Degree,OF,OF,Y,$ACC,Y


In [4]:
segment1_workflow4['WRKFLW_TSK_MAP_DETAILED'].nunique()

134

In [5]:
segment1_workflow4=segment1_workflow4.drop(['WRKFLW_TSK_MAP_GROUPED', 'PATH_CRDATETIME','ITEM_CRUSER','WRKFLW_ORG_TYPE_CRUSER',
                         'STATUS_ORDER','LATEST_STATUS','WRKFLW_PROCESS','QUERY_PART','REFERRAL_ID',
                         'APPLICATION_ID','STU_ID','STU_CITIZEN_CD','STU_CITIZEN_NM','STU_BIRTH_DT',
                         'STU_ABOR_TSI_CD','STU_ABOR_TSI_NM','STU_BIRTH_CNTRY_CD','STU_BIRTH_CNTRY_NM',
                         'STU_CTZN_CNTRY_CD','STU_CTZN_CNTRY_NM','SUBMISSION_METHOD','ENTRY_TYPE',
                         'ASSESSMENT_DECISION_SCORE','ASSESSMENT_DECISION_NM','ASSESSMENT_OUTCOME_NM',
                         'LAST_COUNTRY_EDUCATION','PREV_INST_NM_MAIN','PREV_INST_NM_ALT1','PREV_INST_NM_ALT2',
                        'PREFERENCE_NO','SPK_CD','SPK_FULL_TITLE','ORG_UNIT_CD','AVAIL_ORG_UNIT_NM','SUPPORTING_ORG',
                        'STUDY_MODE_CD','RETURNING_DEFERRAL','CURR_AVAIL_YR','CURR_STUDY_PERIOD','CURR_SPRD_CD',
                        'CURR_SPRD_NM','ORIG_AVAIL_YR','ORIG_STUDY_PERIOD','ORIG_SPRD_CD','ORIG_SPRD_NM',
                         'SPK_CAT_TYPE_CD','BROAD_COURSE_LEVEL','DEGREETYPE','LIABILITY_CATEGORY_CD_RECODE',
                        'ORIG_LIABILITY_CATEGORY_CD_RECODE','OFFER_RELEASED_FG','OFFER_OUTCOME_CD','OFFER_CONDITIONS',
                        'APPLICATION_LINE_ID'], axis=1)
segment1_workflow4.head()

,CASE_ID,WRKFLW_TSK_MAP_DETAILED
0,804599|387970|604118,Release Offer
1,804599|387970|604118,OC - Satisfied - Evidence of course completion
2,804599|387970|604118,Defer
3,844158|432073|669073,Release Offer
4,844158|432073|669073,Manage Acceptance


In [6]:
numOfWorkflowProcesses=segment1_workflow4.groupby('CASE_ID').CASE_ID.count()
numOfWorkflowProcesses.head()

CASE_ID
1000124|358605|558960    2
1000236|334503|523645    2
1000539|334899|524236    2
1000545|335013|524361    3
1000642|335142|524586    1
Name: CASE_ID, dtype: int64

In [7]:
counts = segment1_workflow4['CASE_ID'].value_counts()

In [8]:
counts.head()

1043915|389782|606944    14
1098973|550959|842979    13
813044|372941|581532     13
1107817|476615|731198    12
1074789|435566|673678    12
Name: CASE_ID, dtype: int64

In [474]:
filtered = counts[counts == 14]
seg1_count1=segment1_workflow4[segment1_workflow4['CASE_ID'].isin(filtered.index)]

In [475]:
seg1_count1['CASE_ID'].nunique()

1

In [476]:
seg1_count1.head()

,CASE_ID,WRKFLW_TSK_MAP_DETAILED
3789,1043915|389782|606944,Release Offer
3790,1043915|389782|606944,AR - Satisfied - Further information (1)
3791,1043915|389782|606944,Back to Triage
3792,1043915|389782|606944,Put Triage On-Hold
3793,1043915|389782|606944,OC - Satisfied - Evidence of course completion


In [477]:
y = pd.get_dummies(seg1_count1.WRKFLW_TSK_MAP_DETAILED)
y=y[:0]

In [478]:
y

,AR - Created - Further information (3),AR - Satisfied - Academic results for prior tertiary studies,AR - Satisfied - Further information (1),AR - Satisfied - Further information (3),AR - Satisfied - Overseas Student Health Cover details,Accept,Back to Triage,OC - Satisfied - Evidence of course completion,Put Triage On-Hold,Release Offer,Release Triage Hold


In [479]:
y["CASE_ID"]=""

In [480]:
col_name="CASE_ID"
first_col = y.pop(col_name)

In [481]:
y.insert(0, col_name, first_col)

In [482]:
y['CASE_ID']=seg1_count1['CASE_ID'].unique()

In [483]:
import numpy as np
y = y.replace(np.nan,0, regex=True)
y.head()

,CASE_ID,AR - Created - Further information (3),AR - Satisfied - Academic results for prior tertiary studies,AR - Satisfied - Further information (1),AR - Satisfied - Further information (3),AR - Satisfied - Overseas Student Health Cover details,Accept,Back to Triage,OC - Satisfied - Evidence of course completion,Put Triage On-Hold,Release Offer,Release Triage Hold
0,1043915|389782|606944,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [484]:
dict_1=seg1_count1.to_dict(orient='list')

In [485]:
case_id = dict_1['CASE_ID']

In [486]:
workflow = dict_1['WRKFLW_TSK_MAP_DETAILED']

In [487]:
print(type(workflow))
print(len(workflow))

<class 'list'>
14


In [488]:
from collections import defaultdict

case_wf_dict = defaultdict(list)

for i in range(len(case_id)):
    item = case_id[i]
    case_wf_dict[item].append(workflow[i])
print(len(case_wf_dict))

1


In [489]:
y = y.set_index('CASE_ID')

In [490]:
y.head()

,AR - Created - Further information (3),AR - Satisfied - Academic results for prior tertiary studies,AR - Satisfied - Further information (1),AR - Satisfied - Further information (3),AR - Satisfied - Overseas Student Health Cover details,Accept,Back to Triage,OC - Satisfied - Evidence of course completion,Put Triage On-Hold,Release Offer,Release Triage Hold
CASE_ID,,,,,,,,,,,
1043915|389782|606944,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [491]:
y_col_name = y.columns.values.tolist()
print(y_col_name)

['AR - Created - Further information (3)', 'AR - Satisfied - Academic results for prior tertiary studies', 'AR - Satisfied - Further information (1)', 'AR - Satisfied - Further information (3)', 'AR - Satisfied - Overseas Student Health Cover details', 'Accept', 'Back to Triage', 'OC - Satisfied - Evidence of course completion', 'Put Triage On-Hold', 'Release Offer', 'Release Triage Hold']


In [492]:
for key in case_wf_dict.keys():
    for process in case_wf_dict[key]:
        if process in y_col_name:
            y.loc[key, process] = 1
        else:
            continue

In [493]:
pd.set_option('display.max_rows', None)

In [494]:
def movecol(df, cols_to_move=[], ref_col='', place='After'):
    
    cols = df.columns.tolist()
    if place == 'After':
        seg1 = cols[:list(cols).index(ref_col) + 1]
        seg2 = cols_to_move
    if place == 'Before':
        seg1 = cols[:list(cols).index(ref_col)]
        seg2 = cols_to_move + [ref_col]
    
    seg1 = [i for i in seg1 if i not in seg2]
    seg3 = [i for i in cols if i not in seg1 + seg2]
    
    return(df[seg1 + seg2 + seg3])

In [495]:
y.head()

,AR - Created - Further information (3),AR - Satisfied - Academic results for prior tertiary studies,AR - Satisfied - Further information (1),AR - Satisfied - Further information (3),AR - Satisfied - Overseas Student Health Cover details,Accept,Back to Triage,OC - Satisfied - Evidence of course completion,Put Triage On-Hold,Release Offer,Release Triage Hold
CASE_ID,,,,,,,,,,,
1043915|389782|606944,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [496]:
y = movecol(y, 
             cols_to_move=['Release Offer'], 
             ref_col='AR - Created - Further information (3)',
             place='Before')

In [497]:
y = movecol(y, 
             cols_to_move=['Accept'], 
             ref_col='Release Triage Hold',
             place='After')

In [498]:
y = movecol(y, 
             cols_to_move=['Defer'], 
             ref_col='Release Triage Hold',
             place='After')

KeyError: "['Defer'] not in index"

In [499]:
pd.set_option('display.max_columns',None)

In [500]:
y.shape

(1, 11)

In [501]:
y=y.fillna(0.0)

In [502]:
X=y.iloc[:,0:10].values
Y=y.iloc[:,10].values

In [503]:
#spliting dataset
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size = 0.3,random_state=0)

ValueError: With n_samples=1, test_size=0.3 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [504]:
# Linear SVM
svm = LinearSVC()
svm.fit(x_train, y_train)

ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: 1.0

In [505]:
svm_predict = svm.predict(x_test)
print("accuracy: {}%".format(round(accuracy_score(y_test, svm_predict)*100,2)))

AttributeError: 'LinearSVC' object has no attribute 'coef_'

In [506]:
# Logistic Regression
lr = LogisticRegression()
lr.fit(x_train,y_train)

ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: 1.0

In [507]:
lr_predict=lr.predict(x_test)
print("accuracy: {}%".format(round(accuracy_score(y_test, lr_predict)*100,2)))

AttributeError: 'LogisticRegression' object has no attribute 'coef_'

In [510]:
# K Nearest Neighbors
Knn = KNeighborsClassifier(n_neighbors=1)
Knn.fit(x_train,y_train)

KNeighborsClassifier(n_neighbors=1)

In [511]:
knn_predict=Knn.predict(x_test)
print("accuracy: {}%".format(round(accuracy_score(y_test, knn_predict)*100,2)))

accuracy: 0.0%
